In [2]:

"""
Connect senior:Mainland
"""
import pandas as pd
import json
import requests
def ssql(sql):
    admin_token = 'ae810dc4282e3de074ab08ab631f3461218accf99143b8f3a272e64c1b4c6d0e'
    url = 'http://sensor.wb-intra.com/api/sql/query?token=%s&project=production' % admin_token
    data = {'q': sql, 'format': 'json'}
    req = requests.post(url,data)
    req_dec = req.content.decode()
    try:
        req_json = json.loads('[' + req_dec.replace('\n', ',')[:-1] + ']')
        df_d_id = pd.DataFrame(req_json)
        return df_d_id
    except:
        print(req_dec)

In [3]:
from matplotlib import pyplot as plt
import seaborn as sns
from plotnine import *
from pandas.api.types import CategoricalDtype

In [4]:
df = ssql("""

SELECT t1.distinct_id,
       t1.game_num,
       t1.game_duration,
       t2.pay_num,
       t2.pay_sum,
       t3.friend_num
FROM
  (SELECT distinct_id,
          count(distinct_id)AS game_num,
          sum(duration)/60000 AS game_duration
   FROM events
   WHERE event = 'gameOver'
     AND date BETWEEN CURRENT_DATE() - interval 31 DAY AND CURRENT_DATE() - interval 1 DAY
     AND gameTypeId = 1800
   GROUP BY 1)t1
LEFT JOIN
  (SELECT distinct_id,
          count(distinct_id)AS pay_num,
          sum(pay)AS pay_sum
   FROM events
   WHERE event = 'pay'
     AND date BETWEEN CURRENT_DATE()- interval 31 DAY AND current_date()- interval 1 DAY
   GROUP BY 1)t2 ON t1.distinct_id = t2.distinct_id
LEFT JOIN
  (SELECT distinct_id,
          count(distinct_id)AS friend_num
   FROM EVENTS
   WHERE event = 'addFriendNew'
     AND date BETWEEN CURRENT_DATE()- interval 31 DAY AND current_date() - interval 1 DAY
   GROUP BY 1)t3 ON t1.distinct_id = t3.distinct_id
""")

In [5]:
df

,friend_num,distinct_id,game_num,game_duration,pay_sum,pay_num
0,1.0,120752545,6,24.932,NaN,NaN
1,13.0,115750617,31,212.777,NaN,NaN
2,NaN,125395628,2,5.641,NaN,NaN
3,1.0,129594467,53,177.419,NaN,NaN
4,1.0,111935709,75,287.161,NaN,NaN
...,...,...,...,...,...,...
2245792,14.0,133923333,22,72.879,NaN,NaN
2245793,9.0,137606837,424,1669.950,NaN,NaN
2245794,2.0,133256631,20,88.887,NaN,NaN
2245795,8.0,139438870,9,24.137,NaN,NaN


In [6]:
df_copy = df.copy()
df_copy.fillna(0,inplace=True)
df_copy

,friend_num,distinct_id,game_num,game_duration,pay_sum,pay_num
0,1.0,120752545,6,24.932,0.0,0.0
1,13.0,115750617,31,212.777,0.0,0.0
2,0.0,125395628,2,5.641,0.0,0.0
3,1.0,129594467,53,177.419,0.0,0.0
4,1.0,111935709,75,287.161,0.0,0.0
...,...,...,...,...,...,...
2245792,14.0,133923333,22,72.879,0.0,0.0
2245793,9.0,137606837,424,1669.950,0.0,0.0
2245794,2.0,133256631,20,88.887,0.0,0.0
2245795,8.0,139438870,9,24.137,0.0,0.0


## 进行聚类分析

In [7]:
from sklearn.cluster import KMeans #聚类算法
from sklearn.preprocessing import StandardScaler #标准化
from sklearn.model_selection import GridSearchCV #网格搜索调参 
from sklearn.metrics import silhouette_score #轮廓系数（越大越好）

In [8]:
scaler = StandardScaler()

In [9]:
X = df_copy[['friend_num','game_num','game_duration','pay_sum','pay_num']]

In [10]:
X

,friend_num,game_num,game_duration,pay_sum,pay_num
0,1.0,6,24.932,0.0,0.0
1,13.0,31,212.777,0.0,0.0
2,0.0,2,5.641,0.0,0.0
3,1.0,53,177.419,0.0,0.0
4,1.0,75,287.161,0.0,0.0
...,...,...,...,...,...
2245792,14.0,22,72.879,0.0,0.0
2245793,9.0,424,1669.950,0.0,0.0
2245794,2.0,20,88.887,0.0,0.0
2245795,8.0,9,24.137,0.0,0.0


In [11]:
X = scaler.fit_transform(X)
X

array([[-0.16650939, -0.40689646, -0.35677296, -0.01677967, -0.05523702],
       [ 0.78872104, -0.0558452 ,  0.0980803 , -0.01677967, -0.05523702],
       [-0.24611192, -0.46306466, -0.40348474, -0.01677967, -0.05523702],
       ...,
       [-0.08690685, -0.21030776, -0.20191049, -0.01677967, -0.05523702],
       [ 0.39070836, -0.36477031, -0.35869799, -0.01677967, -0.05523702],
       [ 0.07229822,  0.1407435 ,  0.42307233, -0.01677967, -0.05523702]])

In [54]:
%%time
kmeans = KMeans()
n_clusters = [i for i in range(21,22)]
clf = GridSearchCV(kmeans,param_grid={'n_clusters':n_clusters})
clf.fit(X)

CPU times: user 24min 16s, sys: 1min 8s, total: 25min 25s
Wall time: 4min 4s


GridSearchCV(estimator=KMeans(), param_grid={'n_clusters': [21]})

In [55]:
clf.best_estimator_

KMeans(n_clusters=21)

In [14]:
clf.best_params_

{'n_clusters': 20}

In [56]:
kmeans = clf.best_estimator_

In [57]:
y = kmeans.labels_
y

array([ 0, 17,  0, ...,  0,  0, 17], dtype=int32)

In [58]:
df_copy['label'] = y
df_copy

,friend_num,distinct_id,game_num,game_duration,pay_sum,pay_num,label
0,1.0,120752545,6,24.932,0.0,0.0,0
1,13.0,115750617,31,212.777,0.0,0.0,17
2,0.0,125395628,2,5.641,0.0,0.0,0
3,1.0,129594467,53,177.419,0.0,0.0,17
4,1.0,111935709,75,287.161,0.0,0.0,17
...,...,...,...,...,...,...,...
2245792,14.0,133923333,22,72.879,0.0,0.0,17
2245793,9.0,137606837,424,1669.950,0.0,0.0,2
2245794,2.0,133256631,20,88.887,0.0,0.0,0
2245795,8.0,139438870,9,24.137,0.0,0.0,0


In [59]:
df_copy.groupby(by = ['label'])['distinct_id'].count().sort_values()

label
6           1
5           6
8          12
12         13
20         31
11         35
1          56
18        172
14        197
16        519
4         914
10       1101
13       5059
19       5752
7        6066
2       18415
9       41297
15      51279
3      135658
17     404946
0     1574268
Name: distinct_id, dtype: int64

In [65]:
df_copy.groupby(by = ['label'])[['friend_num','game_num','game_duration','pay_sum','pay_num']].mean().sort_values(by = 'pay_sum').to_clipboard()

In [64]:
df_copy.groupby(by = ['label'])['distinct_id'].count().sort_values().to_clipboard()

In [94]:
cond = df_copy['label'].isin([1,5,6,8,11,12,20])
df_copy[cond]

,friend_num,distinct_id,game_num,game_duration,pay_sum,pay_num,label
10369,914.0,146000167,1,1.870,6.0,1.0,11
37435,3.0,84462008,3,7.762,7508.0,442.0,12
65327,4.0,22468366,1,5.813,24726.0,266.0,1
68634,15.0,26242186,77,346.466,17004.0,228.0,1
94331,74.0,50467971,45,321.616,7393.0,231.0,1
...,...,...,...,...,...,...,...
2144942,9.0,88546364,8,26.112,3728.0,288.0,1
2148272,77.0,124674061,12,52.334,4266.0,245.0,1
2151670,83.0,116021246,34,124.615,10536.0,212.0,1
2174756,43.0,59126114,3,16.323,9608.0,565.0,12


In [95]:
df_mark = df_copy[cond]
df_mark.to_clipboard()

In [22]:
df_c = df_copy[~cond].drop(['label'],axis=1)
df_c

,friend_num,distinct_id,game_num,game_duration,pay_sum,pay_num
0,1.0,120752545,6,24.932,0.0,0.0
2,0.0,125395628,2,5.641,0.0,0.0
5,1.0,105836832,3,9.994,0.0,0.0
6,1.0,98055626,283,1134.685,0.0,0.0
7,0.0,13221187,29,137.785,0.0,0.0
...,...,...,...,...,...,...
2245790,0.0,117185335,5,14.195,0.0,0.0
2245791,7.0,129509894,110,673.524,0.0,0.0
2245793,9.0,137606837,424,1669.950,0.0,0.0
2245794,2.0,133256631,20,88.887,0.0,0.0


### 将离群点复制到粘贴板上

In [23]:
df_mark.to_clipboard()

### 选择聚类的特征，并进行标准化

In [24]:
X_ = df_c[['friend_num','game_num','game_duration','pay_sum','pay_num']]
X_ = scaler.fit_transform(X_)
X_

array([[-0.13346373, -0.32348145, -0.29265985, -0.01923553, -0.03920272],
       [-0.23751437, -0.37442625, -0.33481961, -0.01923553, -0.03920272],
       [-0.13346373, -0.36169005, -0.32530629, -0.01923553, -0.03920272],
       ...,
       [ 0.69894143,  5.0002495 ,  3.30246583, -0.01923553, -0.03920272],
       [-0.02941308, -0.14517467, -0.15288858, -0.01923553, -0.03920272],
       [ 0.59489079, -0.28527286, -0.29439729, -0.01923553, -0.03920272]])

#### 查看特征数目

In [25]:
X_.shape

(1788500, 5)

In [26]:
%%time
kmeans = KMeans()
n_clusters = [i for i in range(9,10)]
clf = GridSearchCV(kmeans,param_grid={'n_clusters':n_clusters})
clf.fit(X_)

CPU times: user 5min 9s, sys: 30.1 s, total: 5min 39s
Wall time: 52.9 s


GridSearchCV(estimator=KMeans(), param_grid={'n_clusters': [9]})

In [27]:
kmeans = clf.best_estimator_
kmeans.labels_

array([0, 0, 0, ..., 8, 0, 0], dtype=int32)

#### 标签数目

In [28]:
kmeans.labels_.shape

(1788500,)

In [29]:
df_c

,friend_num,distinct_id,game_num,game_duration,pay_sum,pay_num
0,1.0,120752545,6,24.932,0.0,0.0
2,0.0,125395628,2,5.641,0.0,0.0
5,1.0,105836832,3,9.994,0.0,0.0
6,1.0,98055626,283,1134.685,0.0,0.0
7,0.0,13221187,29,137.785,0.0,0.0
...,...,...,...,...,...,...
2245790,0.0,117185335,5,14.195,0.0,0.0
2245791,7.0,129509894,110,673.524,0.0,0.0
2245793,9.0,137606837,424,1669.950,0.0,0.0
2245794,2.0,133256631,20,88.887,0.0,0.0


In [30]:
df_c['label'] = kmeans.labels_
df_c
# y_

,friend_num,distinct_id,game_num,game_duration,pay_sum,pay_num,label
0,1.0,120752545,6,24.932,0.0,0.0,0
2,0.0,125395628,2,5.641,0.0,0.0,0
5,1.0,105836832,3,9.994,0.0,0.0,0
6,1.0,98055626,283,1134.685,0.0,0.0,8
7,0.0,13221187,29,137.785,0.0,0.0,0
...,...,...,...,...,...,...,...
2245790,0.0,117185335,5,14.195,0.0,0.0,0
2245791,7.0,129509894,110,673.524,0.0,0.0,2
2245793,9.0,137606837,424,1669.950,0.0,0.0,8
2245794,2.0,133256631,20,88.887,0.0,0.0,0


#### 不同类别下的用户数量

In [31]:
df_c.groupby(by = ['label'])['distinct_id'].count().sort_values()

label
6         54
3        121
1        532
5        659
7       5857
4       6721
8      40472
2     176616
0    1557468
Name: distinct_id, dtype: int64

In [33]:
df_c['label'] = pd.Categorical(df_c['label'])

In [35]:
df_c

,friend_num,distinct_id,game_num,game_duration,pay_sum,pay_num,label
0,1.0,120752545,6,24.932,0.0,0.0,0
2,0.0,125395628,2,5.641,0.0,0.0,0
5,1.0,105836832,3,9.994,0.0,0.0,0
6,1.0,98055626,283,1134.685,0.0,0.0,8
7,0.0,13221187,29,137.785,0.0,0.0,0
...,...,...,...,...,...,...,...
2245790,0.0,117185335,5,14.195,0.0,0.0,0
2245791,7.0,129509894,110,673.524,0.0,0.0,2
2245793,9.0,137606837,424,1669.950,0.0,0.0,8
2245794,2.0,133256631,20,88.887,0.0,0.0,0


In [92]:
df_c.groupby(by  = ['label'])[['pay_sum','friend_num','game_num','game_duration','pay_num']].mean()

,pay_sum,friend_num,game_num,game_duration,pay_num
label,,,,,
0,0.573401,0.895140,8.888326,36.934390,0.027316
1,111.845865,313.507519,285.016917,1328.009998,2.174812
2,2.260854,6.107550,129.521402,649.058341,0.211515
3,26521.966942,45.851240,33.834711,166.625091,80.132231
4,31.079155,96.782770,223.812677,1079.135944,0.917423
5,5642.259484,43.857360,52.136571,276.733495,81.437026
6,12587.333333,34.703704,45.185185,252.530759,364.629630
7,51.295544,22.867680,779.351374,4943.379937,2.220079
8,9.596635,15.373789,325.561153,1848.326864,0.656750


In [80]:
df_c.groupby(by  = ['label'])['distinct_id'].count().to_clipboard()

#### 查看不同类别用户的特征分布

In [93]:
df_c.groupby(by = ['label'])[['friend_num','game_num','game_duration','pay_sum','pay_num']].mean()

,friend_num,game_num,game_duration,pay_sum,pay_num
label,,,,,
0,0.895140,8.888326,36.934390,0.573401,0.027316
1,313.507519,285.016917,1328.009998,111.845865,2.174812
2,6.107550,129.521402,649.058341,2.260854,0.211515
3,45.851240,33.834711,166.625091,26521.966942,80.132231
4,96.782770,223.812677,1079.135944,31.079155,0.917423
5,43.857360,52.136571,276.733495,5642.259484,81.437026
6,34.703704,45.185185,252.530759,12587.333333,364.629630
7,22.867680,779.351374,4943.379937,51.295544,2.220079
8,15.373789,325.561153,1848.326864,9.596635,0.656750


In [53]:
df_c.groupby(by = ['label'])[['friend_num','game_num','game_duration','pay_sum','pay_num']].describe().to_clipboard()

In [77]:
for i in range(9):
    df_c[df_c['label'] == i]['distinct_id'].to_csv('./lable_{}.txt'.format(i),index = False)

In [75]:
# df_e['distinct_id'].to_csv('./test.txt',index=False)

In [76]:
# for i in range(9):
#     print('./label_{}'.format(i))

./label_0
./label_1
./label_2
./label_3
./label_4
./label_5
./label_6
./label_7
./label_8


In [82]:
df_pay = ssql("""
SELECT t1.distinct_id,
       t2.pay_sum
FROM
  (SELECT distinct_id
   FROM events
   WHERE event = 'gameOver'
     AND date BETWEEN '2020-12-26' AND '2021-01-25'
     AND gameTypeId = 1800
   GROUP BY 1)t1
LEFT JOIN
  (SELECT distinct_id,
          sum(pay)AS pay_sum
   FROM events
   WHERE event = 'pay'
     AND date BETWEEN '2020-12-26' AND '2021-01-25'
   GROUP BY 1)t2 ON t1.distinct_id = t2.distinct_id

""")

In [83]:
df_pay

,pay_sum,distinct_id
0,6.0,134087038
1,9.0,139986213
2,108.0,120968285
3,24.0,10038791
4,6.0,125184346
...,...,...
2245792,NaN,118193383
2245793,NaN,135860419
2245794,NaN,117870469
2245795,NaN,145578504


In [85]:
df_pay.fillna(0,inplace=True)

In [88]:
df_pay_copy = df_pay.copy()

In [91]:
X_pay = df_pay_copy['pay_sum']
X_pay

0            6.0
1            9.0
2          108.0
3           24.0
4            6.0
           ...  
2245792      0.0
2245793      0.0
2245794      0.0
2245795      0.0
2245796      0.0
Name: pay_sum, Length: 2245797, dtype: float64

In [96]:
kmeans = KMeans()
n_clusters = [i for i in range(,9)]
clf = GridSearchCV(kmeans,param_grid={'n_clusters':n_clusters})
clf.fit(X)

SyntaxError: invalid syntax (<ipython-input-96-59252cb20bd0>, line 2)

In [111]:
sql_7 = """

SELECT t1.distinct_id,
       t8.friend_num
FROM
  (SELECT distinct_id
   FROM user_group_user_group_tks_label_7
   GROUP BY 1)t1
LEFT JOIN
  (SELECT distinct_id,
          count(distinct_id)AS friend_num
   FROM EVENTS
   WHERE event = 'addFriendNew'
     AND date BETWEEN '2020-12-26' AND '2021-01-25'
   GROUP BY 1)t8 ON t1.distinct_id = t8.distinct_id"""

In [112]:
df_7 = ssql(sql_7)

In [113]:
df_7

,friend_num,distinct_id
0,10.0,11335719
1,3.0,111758295
2,63.0,136958812
3,18.0,121138688
4,12.0,115283351
...,...,...
5853,64.0,81204662
5854,24.0,140360541
5855,55.0,115803484
5856,110.0,63738132


In [114]:
df_7['friend_num'].quantile([0.1,0.25,0.5,0.75,0.9]).to_clipboard()